# Demo: Visualize Streaming Data with InfluxDB & Prospective

## Setup

In [ ]:
# add parent directory to python search path
import sys
sys.path.append('..')       # must run this to import package from parent directory

from time import sleep
from datetime import datetime, timedelta
from generators.smart_grid.new_york_smart_grid import NewYorkSmartGridStreamGenerator
from writers.influxdb_writer import InfluxdbWriter
from writers.file_writer import FileWriter
from utils.logger import logger

# ============================================
# DEMO CONFIGURATION PARAMETERS
# change these parameters to customize the demo
# ============================================
START_TIME = datetime.now() - timedelta(minutes=5)              # start 5 minutes ago
END_TIME = datetime.now() + timedelta(minutes=2)               # end 30 minutes from now
INFLUXDB_URL = 'http://localhost:8086'
INFLUXDB_TOKEN = 'sudo-banana-404'
INFLUXDB_ORG = 'perspective'
INFLUXDB_BUCKET = 'smart_grid'


def run():
    # log demo start message
    logger.info('-' * 50)
    logger.info('Demo: Starting New York Smart Grid demo...')

    # --------------------------------------------
    # DATA GENERATORS & WRITERS

    smart_grid = NewYorkSmartGridStreamGenerator(
        interval=0.250,         # refresh every 0.25 seconds
        nrows=1000,             # number of rows to generate in each batch
        num_stations=32,        # number of stations
        start_time=START_TIME,  # start time
        end_time=END_TIME,      # end time
        loopback=False,         # do not loop back. end the demo after end_time is reached
        )

    # Define the InfluxDB writer
    influx_writer = InfluxdbWriter(
        url=INFLUXDB_URL,
        token=INFLUXDB_TOKEN,
        org=INFLUXDB_ORG,
        bucket=INFLUXDB_BUCKET,
        measurement='new_york_smart_grid',
        timestamp_col='timestamp',
        tag_cols=['station_name', 'latitude', 'longitude'],
        field_cols=['energy_consumption', 'current', 'voltage']
        )
    
    # setup complete
    logger.info('Demo: Setup complete. Starting data generation...')

    # --------------------------------------------
    # MAIN STREAMING LOOP
    smart_grid.add_subscriber(influx_writer.write)
    smart_grid.start()
    # wait for the demo loop to finish or until the user interrupts
    try:
        while smart_grid.is_running():
            sleep(1)
    except KeyboardInterrupt:
        logger.info('Demo: Execution interrupted by the user. Ending demo...')
    finally:
        # Ensure the InfluxDB writer is closed properly
        influx_writer.close()
        del influx_writer
        del smart_grid
        logger.info('Demo: Ended New York Smart Grid demo.')
    


run()